# Dataset Creation

In [1]:
from opik import Opik

In [2]:
# Get or create a dataset
client = Opik()
dataset = client.get_or_create_dataset(name="Lucifer Morning Star")

In [3]:
# Add dataset items to it
dataset.insert([
    {"user_question": "you are poem writer", "expected_output": {"assistant_answer": "poet!"}},
    {"user_question": "What is the capital of India?", "expected_output": {"assistant_answer": "Delhi"}},
])



## Automatic Insert data With LLM invoke

In [5]:
from langchain_ollama import ChatOllama
from opik.integrations.langchain import OpikTracer
from opik import Opik

# Initialize Opik components
client = Opik()
dataset = client.get_or_create_dataset(name="LLM Interactions Dataset")
opik_tracer = OpikTracer(tags=["langchain", "ollama", "qa-recording"])

# Configure Ollama model with tracing
llm = ChatOllama(
    model="Gemma3",
    temperature=0.8,
).with_config({"callbacks": [opik_tracer]})

def ask_and_record(messages):
    """Send message to LLM and automatically record interaction to dataset"""
    # Get the latest user question from messages
    user_question = next(msg[1] for msg in reversed(messages) if msg[0] == "human")
    
    # Get model response
    ai_msg = llm.invoke(messages)
    
    # Record interaction
    dataset.insert([{
        "user_question": user_question,
        "expected_output": {"assistant_answer": ai_msg.content}
    }])
    
    return ai_msg

# Example usage
conversation = [
    ("system", "you are tourist guiderwho helps and guide tourist."),
    ("human", "Give me tour guide about Taj Mahal in agra uttar pradesh?"),
]

# First interaction
response = ask_and_record(conversation)
print("Assistant:", response.content)

# Continue conversation and record subsequent interactions
conversation.append(("ai", response.content))
conversation.append(("human", "Can you guide me for the delhi best places?"))
response = ask_and_record(conversation)
print("Assistant:", response.content)

Assistant: Okay, welcome to Agra, Uttar Pradesh! You've chosen a truly magnificent destination – the Taj Mahal! I’m thrilled to be your guide today. Let’s delve into this incredible monument, a symbol of love and one of the New Seven Wonders of the World.

**A Brief Overview:**

The Taj Mahal is a breathtaking ivory-white marble mausoleum built by the Mughal Emperor Shah Jahan in memory of his beloved wife, Mumtaz Mahal. Construction began in 1632 and was completed around 1648. It's a UNESCO World Heritage Site and a testament to Mughal architecture and artistry.

**Here’s a tour plan we'll follow, and I’ll share some fascinating details along the way:**

**1. Arrival & The Forecourt (Mehtab Bagh - Optional):**

* **First Impressions:** As you approach the Taj Mahal, you’ll be struck by its sheer scale and beauty. It’s a gradual reveal – the closer you get, the more impressive it becomes.
* **Mehtab Bagh (Moon Garden):** For a fantastic, panoramic view of the Taj Mahal, particularly st